## Imports

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Basic
from IPython.display import display

# For OS-agnostic paths
from pathlib import Path

# Plotting
from matplotlib import pyplot as plt
import seaborn as sns
import torch
from torch import nn
import pandas as pd
import numpy as np
sns.set_style("whitegrid")
from copy import deepcopy
import glob, json
from datetime import datetime
import torch
from torch import nn

from torch.utils.data import DataLoader, Dataset
from torch.utils.data import DataLoader, Dataset, TensorDataset


from torchinfo import summary

import mlflow

%cd ..

from src.utils import load_raw_data
from src.plotting import plot_pointcloud, plot_sample_figures
from src.models.HardSphereGAN import GAN
from src.models.StaticScaler import StaticMinMaxScaler

%cd -

plt.set_cmap("viridis")

# Hard spheres model development


First stage: Develop a CNN - based GAN to work with ordered point clouds.'

This notebook is an attempt at the simpler hexagonal and square lattices after slow progress in the full-scale experiment.

In [ ]:

phis = [0.84] # Add more phis here

path = Path("../data/raw/crystal/Sq")
path = Path("../data/raw/crystal/Hex")


files, dataframe, metadata = load_raw_data(path, phi=phis, subpath="disorder-0.2")
# files, dataframe, metadata = load_raw_data(path, phi=phis)

dataframe.sort_index()

In [ ]:
# Hex lattice

N = 1600 
X_box = 41.076212368516387
Y_box = 35.573043402379753   
 
# # Square lattice
# N = 1600
# X_box = 38.225722823651111
# y_box = 38.225722823651111 
# dataframe["r"] = 0.375 # Fixed radius for all data, for square lattice

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StaticMinMaxScaler(
    columns = ["x", "y", "r"],
    maximum = [X_box, Y_box, 2*X_box], # NOTE: Tuned for physical feasibility
    minimum = [-X_box, -Y_box, 0] # NOTE: Tuned for physical feasibility
)

dataframe_scaled = pd.DataFrame(scaler.transform(dataframe), columns=dataframe.columns)

dataframe_scaled.set_index(dataframe.index, inplace=True)

dataframe_scaled = dataframe_scaled.drop(columns=["class"]) # Redundant with r
dataframe_scaled = dataframe_scaled.sort_values(by=["experiment", "sample"])
dataframe_scaled.describe().round(7)

# Plot an example

In [ ]:
sample = dataframe_scaled.copy().query("(sample=='sample-1')").loc[:,["x", "y", "r"]].reset_index(drop=True)
sample = torch.tensor(sample.values).unsqueeze(0)
print(sample.shape)
plot_pointcloud(sample[0], plot_radius=False)

## Build dataset

In [ ]:
dataframe_scaled.head(10)

In [ ]:
## Build dataset
from src.HSDataset import HSDataset

dataset = HSDataset(
    dataframe_scaled.copy(), # Dont use the ordering
    descriptor_list=["phi"],
    synthetic_samples={"rotational": 0, "shuffling": 0, "spatial_offset_static": 0, "spatial_offset_repeats": 0}, 
    downsample=264 / 1600, # 264 is the number of particles in the paper
    keep_r=False
    )
print(dataset[:][0].shape)
print(dataset[:][1].shape)

# Create a function that visualizes the point cloud
plot_pointcloud(dataset[0][1], plot_radius=False)
plot_pointcloud(dataset[-1][1], plot_radius=False)

In [ ]:
plot_pointcloud(dataset[:][1][:,:,:].mean(dim=0), plot_radius=False)

plt.title("Mean Point Cloud")

# Create model

Create a GAN architecture, which creates point clouds $\hat{y}$ based on the descriptor(s) $\hat{X}$ and a random noise vector $\hat{z}$.

In [ ]:
sample_x = dataset[0:32][0].cpu()#.transpose(-1,-2)
sample_y = dataset[0:32][1].cpu()

sample_x_mps = sample_x.to("mps")
sample_y_mps = sample_y.to("mps")

print(sample_x.shape, sample_y.shape)


In [ ]:
# Make a generator model as described in the paper
# paper: https://arxiv.org/pdf/2404.06734

in_features = 64
kernel_size = (3,3) # if 3x3, the output x,y,r will correlate with each other
stride = (1,1)

from src.models.CryinGAN import Generator, CCCGenerator

out_samples = dataset.samples[0].shape[1]

generator_model_2 = CCCGenerator(
    # kernel_size=1, stride=1,
    # rand_features=64, 
    # out_dimensions=2, 
    # fix_r=0.5, 
    # out_samples=out_samples
    clip_output= False,
    fix_r=0.0049,
    kernel_size=[1,1],
    latent_dim=128,
    out_dimensions=2,
    out_samples=264,
    rand_features=64,
    channels_coefficient=1,
    stride=1
    ).to("mps")

print(sample_x.shape)
_out = generator_model_2(sample_x).detach().cpu()
print(_out.shape)

print(summary(generator_model_2, input_data=sample_x, depth=2))

In [ ]:
# Create a discriminator model as described in the paper
from src.models.CryinGAN import CCCGDiscriminator


discriminator_model = CCCGDiscriminator(
  in_samples=264,
  input_channels=2,
  kernel_size=[1,1],
  channels_coefficient=1,
  latent_dim=1056,
  ).to("mps")

print(summary(discriminator_model, input_data=sample_y_mps, depth=2))

## Train the model

In [ ]:
plt.scatter(sample_y[0][:,0].cpu(), sample_y[0][:,1].cpu(), c=list(range(len(sample_y[0][:,0]))), cmap="viridis")
plt.colorbar()

In [ ]:
max_out_samples = N # Max size
downsample = 264 / 1600
fix_r = 0.0049

if downsample:
    out_samples = int(max_out_samples * downsample)
else:
    out_samples = max_out_samples
kernel_size = [1,1]
stride = 1

run_params = {
    "comment": "CCCGenerator",
    "training":{
        "device": "mps" if torch.backends.mps.is_available() else "cpu", # MPS is not supported by PyTorch 2D TransposeConv
        "batch_size": 32,
        "epochs": 5000,
        "early_stopping_patience": 20,
        "early_stopping_headstart": 0,
        "early_stopping_tolerance": 1e-3, # Gradient norm based
        "log_image_frequency": 3,
        "generator_headstart": 0,
        "training_ratio_dg": 3,
        "optimizer_g": {
            "name": "Adam",
            "lr": 0.001, # 0.00005, #0.002, 
            # "hypergrad_lr": 1e-6,
            "weight_decay": 0,
            "betas": [0.5, 0.999]
        },
        "optimizer_d": {
            "name": "Adam",
            "lr": 0.001, #0.002, 
            # "hypergrad_lr": 1e-6,
            "weight_decay": 0,
            "betas": [0.5, 0.999]
        },
        "d_loss":{
            "name": "CryinGANDiscriminatorLoss", # CryinGANDiscriminatorLoss for WaGAN + L1 loss, BCELoss for baseline
            "mu": 1.0, # L1 loss coefficient
        },
        "g_loss":{
            "name": "HSGeneratorLoss",
            "radius_loss": 0,
            "grid_density_loss": 1,
            "gan_loss": 1,
            "distance_loss": 1,
            "physical_feasibility_loss": 0,
            "grid_order_loss": 1,
            "coefficients":{
                "grid_order_k": 4,
                "grid_order_loss": 1,
                "gan_loss": 1,
                "radius_loss": 0,
                "grid_density_loss": 100,
                "physical_feasibility_loss": 0,
                "distance_loss": 100,
            },
        }
    },
    "dataset":{
        "descriptor_list": ["phi"],
        "synthetic_samples":{
            "rotational": 1,
            "shuffling": 0,
            "spatial_offset_static": 0.05,
            "spatial_offset_repeats": 2
            }, # NOTE: Could do subsquares and more rotations.
        "downsample": downsample,
        "keep_r": False
    },
    "generator": {
        "class": "CCCGenerator",
        "kernel_size": kernel_size,
        "stride": stride,
        "channels_coefficient": 1,
        "rand_features": 64,# 513 for one paper, 64 for another,
        "out_dimensions": 2,
        "out_samples": out_samples,
        "latent_dim": 128, # 128 for the papers
        "fix_r": fix_r,
        "clip_output": False
    },
    "discriminator": {
        "class": "CCCGDiscriminator",
        "input_channels": 2, 
        "in_samples": out_samples, 
        "kernel_size": [1,1],
        "channels_coefficient": 3
    },
    "metrics":{
        "packing_fraction": True,
        "packing_fraction_fix_r": fix_r,
        "packing_fraction_box_size": 1,
    }
}

dataset = HSDataset(
    dataframe_scaled.copy(), # Dont use the ordering
    **run_params["dataset"]
    )

sample_x = dataset[0:32][0].cpu()#.transpose(-1,-2)
sample_y = dataset[0:32][1].cpu()

sample_x_mps = sample_x.to("mps")
sample_y_mps = sample_y.to("mps")

print(sample_x.shape, sample_y.shape)
print(dataset.y.shape)
plt.scatter(sample_y[0][:,0].cpu(), sample_y[0][:,1].cpu(), c=list(range(len(sample_y[0]))), cmap="viridis")
plt.colorbar()


test_frac = 0.2

dataset = dataset.to(run_params["training"]["device"])
trainset, testset = torch.utils.data.random_split(dataset, [1-test_frac, test_frac])

sample_x = dataset[0:32][0].cpu()#.transpose(-1,-2)
sample_y = dataset[0:32][1].cpu()

sample_x_mps = sample_x.to("mps")
sample_y_mps = sample_y.to("mps")

print(sample_x.shape, sample_y.shape)

gan = GAN(
    dataset, 
    dataset,# No separate test set
    **run_params
    )

print(summary(gan.generator, input_data=sample_x_mps, depth=2))
print(summary(gan.discriminator, input_data=sample_y_mps, depth=2))

_out = gan.generate(sample_x)[0]

plot_pointcloud(_out, plot_radius=False)
# plt.xlim(0,1)
# plt.ylim(0,1)
10_603_201

Run the training

In [ ]:
import yaml
run_params_yaml = Path("../experiments/1-baseline.yaml")

with open(run_params_yaml, "w") as f:
    yaml.dump(run_params, f, )

# Read the parameters from the yaml to make sure it works
with open(run_params_yaml, "r") as f:
    run_params_yaml = yaml.load(f, Loader=yaml.FullLoader)


# Make sure the parameters are the same
run_params == run_params_yaml

In [ ]:
dataset = HSDataset(
    dataframe_scaled.copy(), # Dont use the ordering
    **run_params_yaml["dataset"]
    )

sample_x = dataset[0:32][0].cpu()#.transpose(-1,-2)
sample_y = dataset[0:32][1].cpu()

sample_x_mps = sample_x.to("mps")
sample_y_mps = sample_y.to("mps")

print(sample_x.shape, sample_y.shape)
print(dataset.y.shape)
plt.scatter(sample_y[0][:,0].cpu(), sample_y[0][:,1].cpu(), c=list(range(len(sample_y[0]))), cmap="viridis")
plt.colorbar()


test_frac = 0.2

dataset = dataset.to(run_params_yaml["training"]["device"])
trainset, testset = torch.utils.data.random_split(dataset, [1-test_frac, test_frac])

sample_x = dataset[0:32][0].cpu()#.transpose(-1,-2)
sample_y = dataset[0:32][1].cpu()

sample_x_mps = sample_x.to("mps")
sample_y_mps = sample_y.to("mps")

print(sample_x.shape, sample_y.shape)

gan = GAN(
    dataset, 
    dataset,# No separate test set
    **run_params_yaml
    )

print(summary(gan.generator, input_data=sample_x_mps, depth=2))
print(summary(gan.discriminator, input_data=sample_y_mps, depth=2))

_out = gan.generate(sample_x)[0]

plot_pointcloud(_out, plot_radius=False)
# plt.xlim(0,1)
# plt.ylim(0,1)
10_603_201

In [ ]:
sample_size = out_samples

gan.train_n_epochs(
    epochs=run_params_yaml["training"]["epochs"],
    batch_size=run_params_yaml["training"]["batch_size"],
    experiment_name=f"Hex lattice, sample size = {sample_size}",
    requirements_file = Path("../top-level-requirements.txt"),
    save_model=True
)

In [ ]:
import mlflow
logged_model = 'runs:/94662023d37747e89a6e769bd9d8aa63/discriminator'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd

data = (dataframe_scaled.query("sample=='sample-1'").loc[:,["x", "y", "r"]].values[::20].reshape(1, 80, 3))
data = data.astype(np.float32)
print(data.shape)

loaded_model.predict(data)

In [ ]:
# NOTE: Alternative generator with diffusion
# generator = CCCGeneratorWithDiffusion(
#     kernel_size=kernel_size,
#     channels_coefficient=1,
#     stride=stride,
#     rand_features=513,
#     out_dimensions=out_dimensions,
#     out_samples=out_samples,
#     latent_dim=256, # initial latent channels
#     fix_r=0.0049,
#     clip_output = False
#     # (
#     #     dataset.y.min(dim=0).values.min(dim=0).values,
#     #     dataset.y.max(dim=0).values.max(dim=0).values
#     #     )
#     ).to("mps")

## Test the discriminator with random data

In [ ]:
# Test the discriminator with random data

# Generate random data
random_data = torch.rand_like(sample_y).to("mps")
random_data = torch.randn_like(sample_y).to("mps")
print(random_data.shape)

plot_pointcloud(random_data[0].cpu().numpy(), plot_radius=False)

# Test the discriminator

output = gan.discriminator(random_data)
print(output.shape)
print("Mean of discriminator output:", output.mean().item())
plt.title(f"Discriminator output: {output[0].item()}")
plt.show()

## Visualize the weights on the first layer

In [ ]:
first_layer_weights.shape

In [ ]:
first_layer_weights.shape

In [ ]:
# Extract the weights from the first layer of the generator
first_layer_weights = gan.generator.model[0].weight.data.cpu().numpy()

max_filters = 64
first_layer_weights = first_layer_weights[:, :max_filters]

# Plot the weights
plt.figure(figsize=(15, 15))
for i in range(1,first_layer_weights.shape[1]):
    plt.subplot(8, 8, i)
    plt.imshow(first_layer_weights[i].reshape(8, 8), cmap='viridis')
    plt.axis('off')
    # Add a global colorbar
    if i == 1:
        plt.colorbar()
        # Relocate the colorbar
        plt.gcf().axes[-1].set_position([0.95, 0.1, 0.03, 0.8])

    plt.title(f'Filter {i}')

plt.suptitle('Weights of the First Layer of the Generator')
plt.show()

In [ ]:
first_layer_weights.shape

In [ ]:
# Extract the weights from the first layer of the generator
first_layer_weights = gan.discriminator.fc_layers[-3].weight.data.cpu().numpy()

# Plot the weights
plt.figure(figsize=(15, 15))
for i in range(1,first_layer_weights.shape[0]):
    plt.subplot(5, 2, i)
    plt.imshow(first_layer_weights[:,i].reshape(5, 2), cmap='viridis')
    plt.axis('off')
    # Add a global colorbar
    if i == 1:
        plt.colorbar()
        # Relocate the colorbar
        plt.gcf().axes[-1].set_position([0.95, 0.1, 0.03, 0.8])

    plt.title(f'Filter {i}')

plt.suptitle('Weights of the First Layer of the Generator')
plt.show()

## 